In [31]:
# make selenium runnable in google colab:https://stackoverflow.com/questions/51046454/how-can-we-use-selenium-webdriver-in-colab-research-google-com 
!pip install selenium
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
wd = webdriver.Chrome('chromedriver',chrome_options=chrome_options)

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:6 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:9 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:13 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:14 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Reading package lists... Done
Reading package lists... Done


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: DeprecationWarning: use options instead of chrome_options
  if sys.path[0] == '':


In [32]:
# Install pip packages in the current Jupyter kernel 
# https://jakevdp.github.io/blog/2017/12/05/installing-python-packages-from-jupyter/
import sys
!{sys.executable} -m pip install pandas
!{sys.executable} -m pip install xlml
!{sys.executable} -m pip install reportlab


ERROR: Could not find a version that satisfies the requirement xlml (from versions: none)
ERROR: No matching distribution found for xlml


Import necessary modules

In [33]:
#import modules 
from selenium import webdriver
import pandas as pd
from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import A4
from reportlab.lib.units import cm
from datetime import datetime

First step is going to the electronic patient file with a web application.  We are gonna use selenium and chrome for that. We first a webbrowser with selenium and go to the right page.

In [34]:
driver = wd
driver.get("https://medicalprogress.dev/patient_file/")

Next we are gonna download the html code and read the table with pandas read_html function

In [35]:
html = driver.page_source
data = pd.read_html(html) #needs lxml to work, make first row the header

Close the browser

In [36]:
driver.close()

We have to clean the data a little bit to make it into a dataframe (list->first item list (table)-> pandas dataframe)

In [37]:
data = data[0]
df = pd.DataFrame(data)

Make a list of variables from the dataframe for printing on pdf

In [38]:
name = df.iloc[0,1]
first_name = df.iloc[1,1]
gender = df.iloc[2,1]
birthdate = df.iloc[3,1]
birth_date_place = df.iloc[4,1]
location = df.iloc[5,1]
address = df.iloc[6,1]
postal_code = df.iloc[7,1]
place = df.iloc[5,1]

Create variables with you personal information

In [39]:
MD = "I.M.You"
specialty = "SuperDoc!"

Create a marking variable for the answers on the pdf

In [40]:
answer = "x"

Get time of death

In [41]:
time_value = input("what is the time of death?")

what is the time of death?3 oclock


Create a canvas for writing on with the file name "deat_certificate.pdf"

In [42]:
canvas = canvas.Canvas
path_pdf = "death_certificate.pdf"
c = canvas(path_pdf)

Create a background variable with first page of decleration form and draw it on the canvas (try to open de file as a check to see if you get the first page of the death decleration form)

In [43]:
background = "https://medicalprogress.dev/patient_file/Form_B1.jpg"
draw_width, draw_height = A4
c.drawImage(background, 0, 0, width=draw_width, height=draw_height)

(1240, 1754)

Write text on form B1

In [44]:
c.drawString(1.9 * cm, 25.7 * cm, place)
c.drawString(1.9 * cm, 23.3 * cm, answer)
c.drawString(1.9 * cm, 20.5 * cm, answer)
if gender == "Male":
    c.drawString(1.9 * cm, 13.3 * cm, answer)
else:
    c.drawString(1.9 * cm, 12.8 * cm, answer)
c.drawString(1.9 * cm, 11.3 * cm, answer)
birthdate_without_dash = birthdate.replace("-", " ")
textobject = c.beginText(4.3 * cm, 10.3 * cm)
textobject.setCharSpace(6.3)
textobject.textLine(birthdate_without_dash)
c.drawText(textobject)
textobject = c.beginText(4.3 * cm, 9.0 * cm)
textobject.setCharSpace(6.3)
textobject.textLine(datetime.now().strftime("%d %m %Y"))
c.drawText(textobject)

Go to next page 

In [45]:
c.showPage()

Write text on form B2

In [46]:
background = "https://medicalprogress.dev/patient_file/Form_B2.jpg"
draw_width, draw_height = A4
c.drawImage(background, 0, 0, width=draw_width, height=draw_height)

(1240, 1754)

In [47]:
c.drawString(3.0 * cm, 12.6 * cm, MD)
c.drawString(3.0 * cm, 11.9 * cm, location)
c.drawString(3.0 * cm, 11.1 * cm, specialty)
c.drawString(3.0 * cm, 10.4 * cm, address)
c.drawString(6.0 * cm, 9.6 * cm, place)
c.drawString(2.8 * cm, 7.0 * cm, answer)
c.drawString(12.5 * cm, 12.6 * cm, "Specialist")
c.drawString(12.5 * cm, 11.9 * cm, "See on the left")
textobject = c.beginText(2.9 * cm, 9.6 * cm)
textobject.setCharSpace(3.8)
textobject.textLine(postal_code)
c.drawText(textobject)
c.drawString(11.5 * cm, 7.3 * cm, datetime.now().strftime("%d %m %y"))
c.showPage()

Write text on form A

In [48]:
background = "https://medicalprogress.dev/patient_file/Form_A.jpg"
draw_width, draw_height = A4
c.drawImage(background, 0, 0, width=draw_width, height=draw_height)

(1240, 1754)

In [49]:
c.drawString(3.5 * cm, 22.9 * cm, MD)
c.drawString(3.5 * cm, 21.8 * cm, place)
c.drawString(5.3 * cm, 19.5 * cm, name)
c.drawString(5.3 * cm, 18.4 * cm, first_name)
c.drawString(7.3 * cm, 17.2 * cm, birthdate + " " + birth_date_place)
c.drawString(7.3 * cm, 15.5 * cm, datetime.now().strftime("%d-%m-%Y"))
c.drawString(10.3 * cm, 15.5 * cm, place)
c.drawString(7.3 * cm, 14.4 * cm, time_value)
c.drawString(3.5 * cm, 4.0 * cm, datetime.now().strftime("%d-%m-%Y"))

Save forms

In [50]:
c.save()

In [51]:
import os
os.remove("death_certificate.pdf")
print("File Removed!")

File Removed!
